# Cloud free mosaics from different sensors

Import dependencies

In [2]:
docker_dir = '/rheil/notebooks/'
### Note: docker_dir is set when you call your docker command. In my case define this directory by calling:
### docker run -d -p 8888:8888 --name gee_demo -v /c/Users/rheil/code_repos/jupyter_gee_demo/:/rheil/notebooks -w /rheil/tylere/docker-debian-python3-scipy-notebook
### the cloud_mask.py file (and any other modules I'd want) are stored in this base directory on the virtual machine

from IPython.display import Image
import ee
ee.Initialize()
import sys 
sys.path.append(docker_dir)
import cloud_mask

Get bounding region

In [3]:
gar_plots = ee.FeatureCollection("ft:14iW585uXib6e89dJyzeNUAXUDpKZECmdvnh4Igbi")
plots = gar_plots
study_area = plots.geometry().bounds()
bounds = study_area.getInfo()['coordinates']

Load Landsat 5 image collection

In [4]:
ic = ee.ImageCollection('LANDSAT/LT5_L1T_TOA_FMASK')

Filter by dates and study area

In [5]:
ic = ic.filterDate('2010-01-01', '2016-06-01')
ic = ic.filterBounds(study_area)

Display first image

In [6]:
img = ee.Image(ic.first())
Image(url=img.select(['B4', 'B5', 'B3']).getThumbUrl({'min':0, 'max':.6, 'region': bounds})) 

Mask clouds, create ndvi band and rename all bands

In [7]:
ic_prepper = cloud_mask.prep_ic('l5', 'toa_fmask')
ic = ic.map(ic_prepper)

Create cloud free mosaic and display

In [8]:
img = ic.qualityMosaic('ndvi')
Image(url=img.select(['nir', 'swir1', 'red']).getThumbUrl({'min':0, 'max':.6, 'region': bounds})) 

Repeat with landsat 8

In [9]:
ic = ee.ImageCollection('LANDSAT/LC8_L1T_TOA_FMASK')
ic_prepper = cloud_mask.prep_ic('l8', 'toa_fmask')
ic = ic.filterDate('2010-01-01', '2016-06-01')
ic = ic.filterBounds(study_area)
ic = ic.map(ic_prepper)
img = ic.qualityMosaic('ndvi')
Image(url=img.select(['nir', 'swir1', 'red']).getThumbUrl({'min':0, 'max':.6, 'region': bounds})) 

Repeat with Sentinel

In [10]:
ic = ee.ImageCollection('COPERNICUS/S2')
ic_prepper = cloud_mask.prep_ic('s2', 'sentinel')
ic = ic.filterDate('2010-01-01', '2016-06-01')
ic = ic.filterBounds(study_area)
ic = ic.map(ic_prepper)
img = ic.qualityMosaic('ndvi')
Image(url=img.select(['nir', 'swir1', 'red']).getThumbUrl({'min':600, 'max': 5000, 'region': bounds})) 